In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Connect to the mendoza database and get payload sizes
db_path = "./data/arkiv-data-mendoza.db"
conn = sqlite3.connect(db_path)

# Query payload sizes (using length of the payload blob)
query = "SELECT length(payload) as payload_size FROM payloads WHERE payload IS NOT NULL"
df = pd.read_sql_query(query, conn)
conn.close()

print(f"Total payloads: {len(df):,}")
print(f"\nPayload size statistics:")
print(df['payload_size'].describe())

In [ ]:
# Create histogram of payload sizes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Linear scale histogram
ax1 = axes[0]
sizes_kb = df['payload_size'] / 1024  # Convert to KB
ax1.hist(sizes_kb, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.set_xlabel('Payload Size (KB)')
ax1.set_ylabel('Number of Payloads')
ax1.set_title('Payload Size Distribution (Linear Scale)')
ax1.axvline(sizes_kb.mean(), color='red', linestyle='--', label=f'Mean: {sizes_kb.mean():.1f} KB')
ax1.axvline(sizes_kb.median(), color='orange', linestyle='--', label=f'Median: {sizes_kb.median():.1f} KB')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Right: Log scale for better visibility of distribution
ax2 = axes[1]
ax2.hist(sizes_kb, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax2.set_xlabel('Payload Size (KB)')
ax2.set_ylabel('Number of Payloads (log scale)')
ax2.set_title('Payload Size Distribution (Log Scale)')
ax2.set_yscale('log')
ax2.axvline(sizes_kb.mean(), color='red', linestyle='--', label=f'Mean: {sizes_kb.mean():.1f} KB')
ax2.axvline(sizes_kb.median(), color='orange', linestyle='--', label=f'Median: {sizes_kb.median():.1f} KB')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('./data/payload_size_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Create size buckets for detailed breakdown
def size_bucket(size_bytes):
    size_kb = size_bytes / 1024
    if size_kb < 1:
        return '< 1 KB'
    elif size_kb < 2:
        return '1-2 KB'
    elif size_kb < 5:
        return '2-5 KB'
    elif size_kb < 10:
        return '5-10 KB'
    elif size_kb < 20:
        return '10-20 KB'
    elif size_kb < 50:
        return '20-50 KB'
    elif size_kb < 100:
        return '50-100 KB'
    else:
        return '> 100 KB'

df['bucket'] = df['payload_size'].apply(size_bucket)

# Order buckets properly
bucket_order = ['< 1 KB', '1-2 KB', '2-5 KB', '5-10 KB', '10-20 KB', '20-50 KB', '50-100 KB', '> 100 KB']
bucket_counts = df['bucket'].value_counts().reindex(bucket_order).fillna(0).astype(int)

print("Payload count by size bucket:")
print("="*40)
for bucket, count in bucket_counts.items():
    pct = count / len(df) * 100
    print(f"{bucket:>12}: {count:>10,} ({pct:>5.1f}%)")
print("="*40)
print(f"{'Total':>12}: {len(df):>10,}")

In [ ]:
# Bar chart of size buckets
fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(bucket_counts.index, bucket_counts.values, edgecolor='black', alpha=0.7, color='steelblue')
ax.set_xlabel('Payload Size Bucket')
ax.set_ylabel('Number of Payloads')
ax.set_title(f'Mendoza Payload Size Distribution ({len(df):,} payloads)')
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', alpha=0.3)

# Add count labels on bars
for bar, count in zip(bars, bucket_counts.values):
    if count > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(bucket_counts.values)*0.01,
                f'{count:,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('./data/payload_size_buckets.png', dpi=150, bbox_inches='tight')
plt.show()